In [1]:
from datasets import load_dataset

e:\EnvForDev\miniconda3\envs\rllm-watermark\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
data_files = "zh-en/validation-00000-of-00001.parquet"


ds = load_dataset("wmt/wmt19", data_files=data_files)

In [26]:
ds["train"]

Dataset({
    features: ['translation'],
    num_rows: 3981
})

In [24]:
dataset = ds["train"].map(
    lambda x: {"zh": x["translation"]["zh"], "en": x["translation"]["en"]}
)

Map: 100%|██████████| 3981/3981 [00:00<00:00, 30917.05 examples/s]


In [25]:
dataset

Dataset({
    features: ['translation', 'zh', 'en'],
    num_rows: 3981
})

In [22]:
# 通过映射函数将结构扁平化
def flatten_translation(batch):
    return {"zh": batch["translation"]["zh"], "en": batch["translation"]["en"]}


# 应用转换（num_proc 启用多进程加速）
dataset = ds["train"].map(flatten_translation, num_proc=4)

Map (num_proc=4): 100%|██████████| 3981/3981 [00:04<00:00, 926.14 examples/s]


In [23]:
dataset

Dataset({
    features: ['translation', 'zh', 'en'],
    num_rows: 3981
})

In [30]:
def extract_translations(example):
    return {
        "en": example["translation"]["en"],
        "zh": example["translation"]["zh"],
    }


transformed_ds = ds["train"].map(
    extract_translations, remove_columns=["translation"]
)

Map: 100%|██████████| 3981/3981 [00:00<00:00, 28266.92 examples/s]


In [31]:
transformed_ds

Dataset({
    features: ['en', 'zh'],
    num_rows: 3981
})

In [ ]:
import os

In [20]:
ds["train"].features["translation"]

Translation(languages=['zh', 'en'], id=None)

In [21]:
import os

# get the pwd
cwd = os.getcwd()
ds["train"].features["translation"].to_json(
    os.path.join(cwd, "validation.jsonl"), lines=True
)

AttributeError: 'Translation' object has no attribute 'to_json'

In [32]:
transformed_ds.to_json(os.path.join(cwd, "validation.jsonl"), lines=True)

Creating json from Arrow format: 100%|██████████| 4/4 [00:00<00:00, 317.85ba/s]


1607686

In [4]:
import os

# get the current dir
current_dir = os.path.dirname(os.path.abspath(__file__))
# get the parent dir
current_dir

NameError: name '__file__' is not defined

In [ ]:
ds.save_to_disk()